# Imports

In [ ]:
import pandas as pd
import csv

# Read File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/final_books.xlsx')
row = [2]
df = df.get('content')

# install hazm

In [ ]:
!pip install hazm

     |████████████████████████████████| 316 kB 8.2 MB/s 
     |████████████████████████████████| 1.4 MB 61.3 MB/s 
     |████████████████████████████████| 233 kB 65.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=a2321b9ddc5d8a14f6f325270e69c2ecfbb304dd4b9875bd9719316b79a54e0d
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154268 sha256=1e192a7eaa5d32929f5c8519d5175a632d0115a49a4ad3f5b08155ff5f6c37f1
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# Normalization

In [ ]:
import hazm

In [ ]:
informalNormalizer = hazm.InformalNormalizer

In [ ]:
normalizer = hazm.Normalizer()

df = df.apply(lambda x: normalizer.character_refinement(normalizer.affix_spacing(str(x))))
df.head()

0     تی دی جیکس یک کشیش، نویسنده، سخنران آمریکایی ...
1     کتاب حاضر که اینک در دسترس خوانندگان محترم قر...
2     دکتر فتح الله بینا تحصیلات خود را در رشته پزش...
3     مجموعه پیش رو مطالبیست که در گروه Evolution ت...
4     کتاب آسمان پرستاره نوشته‌ی کتی هایدن، به زبان...
Name: content, dtype: object

In [ ]:
df = df.apply(lambda x: normalizer.normalize(str(x)))
df.head()

0     تی دی جیکس یک کشیش، نویسنده، سخنران آمریکایی ...
1     کتاب حاضر که اینک در دسترس خوانندگان محترم قر...
2     دکتر فتح الله بینا تحصیلات خود را در رشته پزش...
3     مجموعه پیش رو مطالبیست که در گروه Evolution ت...
4     کتاب آسمان پرستاره نوشته‌ی کتی هایدن، به زبان...
Name: content, dtype: object

In [ ]:
!git clone https://github.com/kharazi/persian-stopwords.git

Cloning into 'persian-stopwords'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 51 (delta 7), reused 5 (delta 5), pack-reused 42
Unpacking objects: 100% (51/51), done.


In [ ]:
word_tokenize = hazm.word_tokenize
token_df = pd.DataFrame()
token_df['tokens'] =df.apply(lambda x: word_tokenize(x))
token_df

,tokens
0,"[تی, دی, جیکس, یک, کشیش, ،, نویسنده, ،, سخنران..."
1,"[کتاب, حاضر, که, اینک, در, دسترس, خوانندگان, م..."
2,"[دکتر, فتح, الله, بینا, تحصیلات, خود, را, در, ..."
3,"[مجموعه, پیش, رو, مطالبیست, که, در, گروه, Evol..."
4,"[کتاب, آسمان, پرستاره, نوشته‌ی, کتی, هایدن, ،,..."
...,...
2819,[nan]
2820,[nan]
2821,"[کتابی, که, می‌خواهید, مطالعه, کنید, به, صورت,..."
2822,[nan]


In [ ]:
chars = []
with open('/content/persian-stopwords/chars', 'r') as f:
    for line in f:
      chars.append(line.strip())

In [ ]:
def remove_stopwords(tokens):
  sw = hazm.utils.stopwords_list()
  return [item for item in tokens if item not in chars and item not in sw]

In [ ]:
token_df['without-stop-word'] = token_df['tokens'].apply(lambda x: remove_stopwords(x))
token_df['without-stop-word']

0       [تی, دی, جیکس, کشیش, نویسنده, سخنران, آمریکایی...
1       [کتاب, حاضر, اینک, دسترس, خوانندگان, محترم, قر...
2       [دکتر, فتح, الله, بینا, تحصیلات, رشته, پزشکی, ...
3       [مجموعه, مطالبیست, گروه, Evolution, توسط, اینج...
4       [کتاب, آسمان, پرستاره, نوشته‌ی, کتی, هایدن, زب...
                              ...                        
2819                                                [nan]
2820                                                [nan]
2821    [کتابی, می‌خواهید, مطالعه, صورت, فیلم, نامه, م...
2822                                                [nan]
2823                                                [nan]
Name: without-stop-word, Length: 2824, dtype: object

# lemmatizer

In [ ]:
def stem(tokens):
  stemmer = hazm.Stemmer()
  return [stemmer.stem(item) for item in tokens]

In [ ]:
def lem(tokens):
  lemmatizer = hazm.Lemmatizer()
  return [lemmatizer.lemmatize(x) for x in tokens]

In [ ]:
token_df['stemmed'] = token_df['without-stop-word'].apply(lambda x: stem(x))
token_df['stemmed'].head()

0    [ت, د, جیکس, کش, نویسنده, سخنر, آمریکا, سمینار...
1    [کتاب, حاضر, اینک, دسترس, خوانندگ, محتر, قرار,...
2    [دک, فتح, الله, بینا, تحصیل, رشته, پزشک, دنبال...
3    [مجموعه, مطالبیس, گروه, Evolution, توسط, اینجا...
4    [کتاب, آس, پرستاره, نوشته, کت, هایدن, زبان, سا...
Name: stemmed, dtype: object

In [ ]:
token_df['lemmetized'] = token_df['stemmed'].apply(lambda x: lem(x))
token_df['lemmetized'].head()

0    [ت, د, جیکس, کش, نویسنده, سخنر, آمریکا, سمینار...
1    [کتاب, حاضر, اینک, دسترس, خوانندگ, محتر, قرار,...
2    [دک, فتح, الله, بینا, تحصیل, رشته, پزشک, دنبال...
3    [مجموعه, مطالبیس, گروه, Evolution, توسط, اینجا...
4    [کتاب, آس, پرستاره, نوشته, کت, هایدن, زبان, سا...
Name: lemmetized, dtype: object

In [ ]:
for row in token_df['without-stop-word']:
  print(lem(row))

In [ ]:
token_df['lem'] = lem(item for item in token_df['without-stop-word'])

# Inverted Index 
after lemmatization


In [ ]:
docs = token_df["lemmetized"].tolist()
index_map = {}
for doc in docs:
  doc_index = docs.index(doc) + 1
  doc = list(set(doc))
  for token in doc:
    if token not in index_map.keys():
      index_map[token] = [doc_index,]
    else: 
      index_map[token].append(doc_index)

write output of inverted index to a csv file

In [50]:
with open("/content/II_lem_output.csv", 'w', newline='') as output_file:
  csvwriter = csv.writer(output_file)
  for key in index_map.keys():
    row = []
    row.append(key)
    for i in index_map[key]:
      row.append(i)
    csvwriter.writerow(row)

# Inverted Index
after stemming

In [48]:
docs = token_df["stemmed"].tolist()
index_map = {}
for doc in docs:
  doc_index = docs.index(doc) + 1
  doc = list(set(doc))
  for token in doc:
    if token not in index_map.keys():
      index_map[token] = [doc_index,]
    else: 
      index_map[token].append(doc_index)

write output of inverted index to a csv file

In [49]:
with open("/content/II_stm_output.csv", 'w', newline='') as output_file:
  csvwriter = csv.writer(output_file)
  for key in index_map.keys():
    row = []
    row.append(key)
    for i in index_map[key]:
      row.append(i)
    csvwriter.writerow(row)